<a href="https://colab.research.google.com/github/JingchenYan1/Intro-to-ML/blob/main/Homework7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # [N,16,16,16]
        x = self.pool(torch.relu(self.conv2(x)))  # [N,32,8,8]
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(loader.dataset)
    return epoch_loss

def test(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100. * correct / total
    return acc

num_epochs = 200
start_time = time.time()
for epoch in range(num_epochs):
    train_loss = train(model, trainloader, optimizer, criterion, device)
    acc = test(model, testloader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Acc: {acc:.2f}%")

end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time:.2f}s after {num_epochs} epochs")


100%|██████████| 170M/170M [00:13<00:00, 12.7MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/200], Loss: 2.1171, Test Acc: 31.88%
Epoch [2/200], Loss: 1.8636, Test Acc: 36.64%
Epoch [3/200], Loss: 1.7203, Test Acc: 41.67%
Epoch [4/200], Loss: 1.6136, Test Acc: 45.96%
Epoch [5/200], Loss: 1.5279, Test Acc: 47.43%
Epoch [6/200], Loss: 1.4715, Test Acc: 49.69%
Epoch [7/200], Loss: 1.4217, Test Acc: 52.02%
Epoch [8/200], Loss: 1.3894, Test Acc: 52.67%
Epoch [9/200], Loss: 1.3574, Test Acc: 54.55%
Epoch [10/200], Loss: 1.3236, Test Acc: 54.79%
Epoch [11/200], Loss: 1.2952, Test Acc: 57.39%
Epoch [12/200], Loss: 1.2676, Test Acc: 58.71%
Epoch [13/200], Loss: 1.2338, Test Acc: 59.59%
Epoch [14/200], Loss: 1.2109, Test Acc: 60.42%
Epoch [15/200], Loss: 1.1879, Test Acc: 60.33%
Epoch [16/200], Loss: 1.1569, Test Acc: 62.61%
Epoch [17/200], Loss: 1.1384, Test Acc: 63.50%
Epoch [18/200], Loss: 1.1162, Test Acc: 63.76%
Epoch [19/200], Loss: 1.0979, Test Acc: 64.61%
Epoch [20/200], Loss: 1.082

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

class DeeperCNN(nn.Module):
    def __init__(self):
        super(DeeperCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # 新增一层卷积

        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = DeeperCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    for inputs, targets in loader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()*inputs.size(0)
    epoch_loss = running_loss/len(loader.dataset)
    return epoch_loss

def test(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    acc = 100.*correct/total
    return acc

num_epochs = 200
for epoch in range(num_epochs):
    train_loss = train(model, trainloader, optimizer, criterion, device)
    test_acc = test(model, testloader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {train_loss:.4f}, Test Acc: {test_acc:.2f}%")


Files already downloaded and verified
Files already downloaded and verified
Epoch [1/200] Loss: 2.2719, Test Acc: 23.65%
Epoch [2/200] Loss: 2.0543, Test Acc: 29.89%
Epoch [3/200] Loss: 1.9233, Test Acc: 34.43%
Epoch [4/200] Loss: 1.8155, Test Acc: 37.96%
Epoch [5/200] Loss: 1.7106, Test Acc: 41.44%
Epoch [6/200] Loss: 1.6264, Test Acc: 45.39%
Epoch [7/200] Loss: 1.5519, Test Acc: 47.99%
Epoch [8/200] Loss: 1.4896, Test Acc: 50.06%
Epoch [9/200] Loss: 1.4444, Test Acc: 52.78%
Epoch [10/200] Loss: 1.4097, Test Acc: 53.98%
Epoch [11/200] Loss: 1.3708, Test Acc: 54.06%
Epoch [12/200] Loss: 1.3400, Test Acc: 56.69%
Epoch [13/200] Loss: 1.3101, Test Acc: 56.72%
Epoch [14/200] Loss: 1.2830, Test Acc: 58.21%
Epoch [15/200] Loss: 1.2479, Test Acc: 59.41%
Epoch [16/200] Loss: 1.2273, Test Acc: 59.23%
Epoch [17/200] Loss: 1.1989, Test Acc: 60.96%
Epoch [18/200] Loss: 1.1773, Test Acc: 60.68%
Epoch [19/200] Loss: 1.1601, Test Acc: 62.04%
Epoch [20/200] Loss: 1.1368, Test Acc: 61.54%
Epoch [21/200

In [7]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)

        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 128, num_blocks[3], stride=2)

        self.linear = nn.Linear(128, num_classes)

    def _make_layer(self, block, planes, num_block, stride):
        layers = []
        layers.append(block(self.in_planes, planes, stride))
        self.in_planes = planes
        for i in range(1, num_block):
            layers.append(block(planes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        out = torch.mean(out, dim=[2,3])  # Global Average Pooling
        out = self.linear(out)
        return out

model = ResNet(BasicBlock, [1,1,1,1], num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 200
start_time = time.time()
for epoch in range(num_epochs):
    train_loss = train(model, trainloader, optimizer, criterion, device)
    acc = test(model, testloader, device)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Test Acc: {acc:.2f}%")

end_time = time.time()
training_time = end_time - start_time
print(f"ResNet-10 Training time: {training_time:.2f}s after {num_epochs} epochs")


Epoch [1/200], Loss: 1.9224, Test Acc: 38.20%
Epoch [2/200], Loss: 1.6187, Test Acc: 42.93%
Epoch [3/200], Loss: 1.4635, Test Acc: 49.48%
Epoch [4/200], Loss: 1.3512, Test Acc: 52.47%
Epoch [5/200], Loss: 1.2649, Test Acc: 51.65%
Epoch [6/200], Loss: 1.1940, Test Acc: 57.89%
Epoch [7/200], Loss: 1.1365, Test Acc: 59.76%
Epoch [8/200], Loss: 1.0772, Test Acc: 59.65%
Epoch [9/200], Loss: 1.0308, Test Acc: 62.15%
Epoch [10/200], Loss: 0.9854, Test Acc: 66.41%
Epoch [11/200], Loss: 0.9476, Test Acc: 66.48%
Epoch [12/200], Loss: 0.9103, Test Acc: 67.33%
Epoch [13/200], Loss: 0.8804, Test Acc: 66.73%
Epoch [14/200], Loss: 0.8499, Test Acc: 69.14%
Epoch [15/200], Loss: 0.8257, Test Acc: 70.59%
Epoch [16/200], Loss: 0.8018, Test Acc: 69.37%
Epoch [17/200], Loss: 0.7756, Test Acc: 72.07%
Epoch [18/200], Loss: 0.7586, Test Acc: 71.08%
Epoch [19/200], Loss: 0.7302, Test Acc: 72.42%
Epoch [20/200], Loss: 0.7188, Test Acc: 72.54%
Epoch [21/200], Loss: 0.7021, Test Acc: 75.04%
Epoch [22/200], Loss: 